In [25]:
import pandas as pd
import re
import os
from dotenv import load_dotenv
from github import Github
from tqdm import tqdm
import csv
import requests



In [5]:
org_name = "Rdatatable"
repo_name = "data.table"

reponame_noperiod = repo_name.replace(".", "")
reponame_noperiod = reponame_noperiod.replace("_", "")
reponame_noperiod = reponame_noperiod.lower()

In [ ]:
g = Github(secret)
org = g.get_organization(org_name)
repo = org.get_repo(repo_name)
g.rate_limiting

(4996, 5000)

In [21]:
file_path_comments = f'Files/{org_name.lower()}_{reponame_noperiod}_pull_request_comments.xlsx'

file_path = f'Files/{org_name.lower()}_{reponame_noperiod}_pull_requests.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    existing_pull_df = pd.read_excel(file_path, engine='openpyxl')
    print("Pull Request file exist")
else:
    existing_pull_df = pd.DataFrame()
    print("Error:Pull Request file does not exist ", file_path)

if os.path.exists(file_path_comments):
    existing_pull_df_comments = pd.read_excel(file_path_comments, engine='openpyxl')
    print("Pull Request Comments file exist")
else:
    existing_pull_df_comments = pd.DataFrame()
    print("Error:Pull Request Comments file does not exist ",file_path_comments)

# Define the file path
file_path_comments = f'Files/{org_name.lower()}_{reponame_noperiod}_issues_comments.xlsx'

file_path = f'Files/{org_name.lower()}_{reponame_noperiod}_issues.xlsx'

# Check if the file exists
if os.path.exists(file_path):
    existing_issues_df = pd.read_excel(file_path, engine='openpyxl')
    print("Issues Request file exists")
else:
    existing_issues_df = pd.DataFrame()
    print("Error: Issues Request file does not exist ", file_path )
if os.path.exists(file_path_comments):
    existing_issues_df_comments = pd.read_excel(file_path_comments, engine='openpyxl')
    print("Issues Request Comments file exists")
else:
    existing_issues_df_comments = pd.DataFrame()
    print("Error: Issues Comments file does not exist ", file_path_comments)

existing_pull_df['created_by'] = existing_pull_df['created_by'].apply(lambda x: x.split('/')[-1])
existing_issues_df['created_by'] = existing_issues_df['created_by'].apply(lambda x: x.split('/')[-1])
existing_pull_df_comments['created_by'] = existing_pull_df_comments['created_by'].apply(lambda x: re.search(r'login="([^"]+)"', x).group(1))
existing_issues_df_comments['created_by'] = existing_issues_df_comments['created_by'].apply(lambda x: re.search(r'login="([^"]+)"', x).group(1))

Pull Request file exist
Pull Request Comments file exist
Issues Request file exists
Issues Request Comments file exists


In [22]:
def get_user_data(df, column_name):
    if column_name in df.columns:
        user_counts = df[column_name].value_counts().reset_index()
        user_counts.columns = [column_name, 'Frequency']
        return user_counts
    else:
        print(f"Column '{column_name}' not found in DataFrame")
        return pd.DataFrame()

# Get unique users and their frequency for pull requests
pull_user_frequency = get_user_data(existing_pull_df, 'created_by')

# Get unique users and their frequency for pull request comments
pull_comments_user_frequency = get_user_data(existing_pull_df_comments, 'created_by')

# Get unique users and their frequency for issues
issues_user_frequency = get_user_data(existing_issues_df, 'created_by')

# Get unique users and their frequency for issues comments
issues_comments_user_frequency = get_user_data(existing_issues_df_comments, 'created_by')

# Optionally, save the results to Excel files
pull_user_frequency.to_excel(f'Files/{org_name.lower()}_{reponame_noperiod}_pull_user_frequency.xlsx', index=False)
pull_comments_user_frequency.to_excel(f'Files/{org_name.lower()}_{reponame_noperiod}_pull_comments_user_frequency.xlsx', index=False)
issues_user_frequency.to_excel(f'Files/{org_name.lower()}_{reponame_noperiod}_issues_user_frequency.xlsx', index=False)
issues_comments_user_frequency.to_excel(f'Files/{org_name.lower()}_{reponame_noperiod}_issues_comments_user_frequency.xlsx', index=False)


# Merge the dataframes on 'created_by' column
merged_df = pull_user_frequency.merge(pull_comments_user_frequency, on='created_by', how='outer', suffixes=('_pulls', '_pull_comments'))
merged_df = merged_df.merge(issues_user_frequency, on='created_by', how='outer')
merged_df = merged_df.merge(issues_comments_user_frequency, on='created_by', how='outer', suffixes=('_issues', '_issues_comments'))

# Rename columns for clarity
merged_df.columns = ['created_by', 'Frequency_pulls', 'Frequency_pull_comments', 'Frequency_issues', 'Frequency_issues_comments']

# Fill NaN values with 0
merged_df = merged_df.fillna(0)

# Save the merged dataframe to an Excel file
merged_df.to_excel(f'Files/{org_name.lower()}_{reponame_noperiod}_user_activity_summary.xlsx', index=False)


print("Merged DataFrame:")
print(merged_df)

Merged DataFrame:
       created_by  Frequency_pulls  Frequency_pull_comments  Frequency_issues  \
0         0xorial              0.0                      0.0               0.0   
1         123saga              0.0                      0.0               1.0   
2           2005m              9.0                     56.0               6.0   
3     AKRosenblad              0.0                      0.0               1.0   
4        APLevine              0.0                      0.0               1.0   
...           ...              ...                      ...               ...   
1741   zlskidmore              0.0                      0.0               0.0   
1742    zoushucai              0.0                      0.0               0.0   
1743     zuoliang              0.0                      0.0               1.0   
1744         zwep              0.0                      0.0               1.0   
1745       zx8754              0.0                      0.0               5.0   

      Fre

In [ ]:


# Read the list of users from the CSV file
users = merged_df['created_by'].tolist()

def fetch_user_events(username, token):
    url = f"https://api.github.com/users/{username}/events"
    headers = {'Authorization': f'token {token}'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        events = response.json()
        important_events = []
        
        for event in events:
            event_type = event['type']
            
            # Filter for PushEvent, IssuesEvent, and IssueCommentEvent
            if event_type in ['PushEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestEvent']:
                important_event = {
                    'username': username,
                    'type': event_type,
                    'created_at': event['created_at'],
                    'repo': event['repo']['name'],
                    'public': event['public']
                }
                
                # Add additional details based on event type
                if event_type == 'PushEvent':
                    important_event['commit_count'] = len(event['payload']['commits'])
                    important_event['branch'] = event['payload']['ref'].split('/')[-1]  # Extract branch name
                
                elif event_type == 'IssuesEvent':
                    important_event['action'] = event['payload']['action']  # e.g., opened, closed
                    important_event['issue_id'] = event['payload']['issue']['id']
                    important_event['issue_title'] = event['payload']['issue']['title']
                
                elif event_type == 'IssueCommentEvent':
                    important_event['comment_id'] = event['payload']['comment']['id']
                    important_event['issue_id'] = event['payload']['issue']['id']
                    important_event['issue_title'] = event['payload']['issue']['title']
                
                elif event_type == 'PullRequestEvent':
                    important_event['action'] = event['payload']['action']  # e.g., opened, closed, merged
                    important_event['pr_id'] = event['payload']['pull_request']['id']
                    important_event['pr_title'] = event['payload']['pull_request']['title']
                    important_event['pr_state'] = event['payload']['pull_request']['state']
                    important_event['pr_merged'] = event['payload']['pull_request']['merged']
                    important_event['pr_commits'] = event['payload']['pull_request']['commits']
                    important_event['pr_additions'] = event['payload']['pull_request']['additions']
                    important_event['pr_deletions'] = event['payload']['pull_request']['deletions']
                
                important_events.append(important_event)
        
        return important_events
    else:
        print(f"Failed to fetch events for {username}: {response.status_code}")
        return []


In [45]:

def fetch_user_events(username, token):
    url = f"https://api.github.com/users/{username}/events"
    headers = {'Authorization': f'token {token}'} if token else {}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        events = response.json()
        filtered_events = []
        
        for event in events:
            event_type = event['type']
            if event_type in ['PushEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestEvent']:
                filtered_events.append({
                    'username': username,
                    'type': event_type,
                    'created_at': event['created_at'],
                    'repo': event['repo']['name'],
                    'public': event['public']
                })
        
        return filtered_events
    else:
        print(f"Failed to fetch events: {response.status_code}")
        return []

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)


events = fetch_user_events("jangorecki"  , secret)

if events:
    save_to_excel(events, f"{username}_events.xlsx")
    print(f"Data saved to {username}_events.xlsx")
else:
    print("No events found or failed to fetch events.")

Data saved to arunsrinivasan_events.xlsx


In [44]:
users_0_list = fetch_user_events("jangorecki"  , secret)

# Convert users_0_list to a DataFrame
users_0_df = pd.DataFrame(users_0_list)

# Define the file path for the new Excel file
users_0_file_path = f'Files/{users[0]}_events.xlsx'

# Save the DataFrame to an Excel file
users_0_df.to_excel(users_0_file_path, index=False)

print(f"Events for {users[0]} saved to {users_0_file_path}")



Events for 0xorial saved to Files/0xorial_events.xlsx


In [37]:
users_0_list

[{'username': '0xorial',
  'type': 'IssueCommentEvent',
  'created_at': '2025-01-23T20:44:48Z',
  'repo': 'rust-lang/rust-analyzer',
  'public': True,
  'comment_id': 2610980172,
  'issue_id': 2677614346,
  'issue_title': '"Go To References" for `PartialOrd` functions should find comparison operators'},
 {'username': '0xorial',
  'type': 'IssueCommentEvent',
  'created_at': '2025-01-21T11:58:06Z',
  'repo': 'element-hq/element-desktop',
  'public': True,
  'comment_id': 2604527765,
  'issue_id': 2796031806,
  'issue_title': 'None of the old messages are decrypted.'},
 {'username': '0xorial',
  'type': 'IssuesEvent',
  'created_at': '2025-01-21T11:42:13Z',
  'repo': 'orvium/orvium',
  'public': True,
  'action': 'opened',
  'issue_id': 2801585858,
  'issue_title': 'Project status clarification'},
 {'username': '0xorial',
  'type': 'IssueCommentEvent',
  'created_at': '2025-01-20T15:40:40Z',
  'repo': 'element-hq/element-desktop',
  'public': True,
  'comment_id': 2602737255,
  'issue_id

In [ ]:
# Fetch events for each user and save to a new file
for user in users:
    events = fetch_user_events(user, secret)
    if events:
        user_file_path = f'Files/{user}_events.json'
        with open(user_file_path, 'w') as user_file:
            json.dump(events, user_file)
        print(f"Events for {user} saved to {user_file_path}")
    else:
        print(f"No events found for {user}")